![Photo by Stephen Phillips - Hostreviews.co.uk on UnSplash](https://cf.bstatic.com/xdata/images/hotel/max1024x768/408003083.jpg?k=c49b5c4a2346b3ab002b9d1b22dbfb596cee523b53abef2550d0c92d0faf2d8b&o=&hp=1){fig-align="center" width=50%}


# Import data

In [1]:
import time
from pathlib import Path

import numpy as np
import pandas as pd
from data import utils
from lets_plot import *
from lets_plot.mapping import as_discrete

LetsPlot.setup_html()

**Goal**:
- Determine the essential preprocessing actions required post-web scraping and prior to data uploading into a database

# Data Pre-cleaning steps

In the upcoming section, we will delve into fundamental post-web scraping procedures. While scraping, we acquired approximately 50 features, rendering our dataset information-rich. However, there's significant work ahead due to the data's lack of cleanliness. This entails tasks such as dtype conversion, column parsing to extract numerical values, and the transformation of Boolean values into binary variables.  
Here is the dataset we have gathered through web scraping:

In [7]:
for filename in utils.Configuration.RAW_DATA_PATH.glob("*.gzip"):
    if "data" in filename.stem:
        df = pd.read_parquet(filename)
print(df.shape)
df.head().style.set_sticky(axis=0)

(60, 50)


,Available as of,Construction year,Building condition,Street frontage width,Number of frontages,Covered parking spaces,Outdoor parking spaces,Surroundings type,Living area,Living room surface,Kitchen type,Kitchen surface,Bedrooms,Bedroom 1 surface,Bedroom 2 surface,Bedroom 3 surface,Bathrooms,Toilets,Basement,Furnished,Surface of the plot,Width of the lot on the street,Connection to sewer network,"Gas, water & electricity",Garden surface,TV cable,Primary energy consumption,Energy class,Reference number of the EPC report,CO₂ emission,Yearly theoretical total energy consumption,Heating type,Double glazing,Planning permission obtained,Subdivision permit,Possible priority purchase right,Proceedings for breach of planning regulations,Flood zone type,Latest land use designation,Price,Cadastral income,Tenement building,Address,External reference,day_of_retrieval,ad_url,Website,As built plan,Office,Dining room
0,After signing the deed,1971,Just renovated,12 m,4,2,5,"Living area (residential, urban or rural)",197 m² square meters,47 m² square meters,Hyper equipped,16 m² square meters,4,18 m² square meters,13 m² square meters,9 m² square meters,1,2,Yes,No,992 m² square meters,20 m meters,Connected,Yes,485 m² square meters,Yes,371 kWh/m² kilowatt hour per square meters,D,1-RES,9784 kg CO₂/m²,Not specified,Gas,Yes,Yes,No,Yes,No,Non flood zone,"Living area (residential, urban or rural)","€ 465,000 465000 €","€ 1,279 1279 €",No,Stationstraat 30 9600 - Ronse,5411439,2023-09-23 17:11:29.870179,https://www.immoweb.be/en/classified/villa/for-sale/ronse/9600/10838435,None,None,None,None
1,After signing the deed,1949,To renovate,18 m,3,1,None,Isolated,139 m² square meters,10 m² square meters,Semi equipped,13 m² square meters,2,17 m² square meters,11 m² square meters,None,1,1,Yes,No,413 m² square meters,18 m meters,Connected,None,315 m² square meters,Yes,699 kWh/m² kilowatt hour per square meters,G,20230303013078,173 kg CO₂/m²,96913 kWh/year,Fuel oil,Yes,None,No,No,No,Non flood zone,"Living area (residential, urban or rural)","Make an offer starting from € 150,000 Make an offer starting from 150000 €",€ 689 689 €,No,Rue de la Wallonie 2A 4680 - Oupeye,5534704,2023-09-23 17:11:29.995637,https://www.immoweb.be/en/classified/house/for-sale/esneux%20tilff/4130/10838582,http://www.nigel-immo.be,None,Yes,Yes
2,After signing the deed,1920,Good,5.5 m,2,None,None,Urban,200 m² square meters,26 m² square meters,Installed,13 m² square meters,3,22 m² square meters,21 m² square meters,13 m² square meters,1,1,Yes,No,136 m² square meters,None,Connected,Yes,55 m² square meters,Yes,239 kWh/m² kilowatt hour per square meters,C,2281999,9544 kg CO₂/m²,Not specified,Gas,Yes,None,No,No,No,Non flood zone,"Living area (residential, urban or rural)","€ 529,000 529000 €",€ 917 917 €,No,Sint-Denijslaan 1 9000 - Gent,5531386,2023-09-23 17:11:32.203024,https://www.immoweb.be/en/classified/mansion/for-sale/gent/9000/10835799,http://www.immodavinci.be,No,Yes,None
3,After signing the deed,1937,To renovate,8 m,2,None,None,Urban,230 m² square meters,None,Semi equipped,None,3,None,None,None,None,2,None,No,154 m² square meters,8 m meters,Connected,Yes,None,Yes,496 kWh/m² kilowatt hour per square meters,E,20230804-0002955788-KNR-1,Not specified,Not specified,Gas,Yes,Yes,No,No,No,None,"Living area (residential, urban or rural)","€ 245,000 245000 €","€ 1,623 1623 €",No,Hoogstraat 20 9340 - Lede,5535368,2023-09-23 17:11:34.716784,https://www.immoweb.be/en/classified/house/for-sale/sint-lievens-houtem/9520/10835368,http://www.immoderas.be,None,Yes,None
4,Depending on the tenant,1900,Good,6.5 m,2,None,None,Urban,360 m² square meters,34 m² square meters,Installed,None,8,20 m² square meters,18 m² square meters,18 m² square meters,5,6,Yes,No,173 m² square meters,6.5 m meters,Connected,Yes,36 m² square meters,Yes,249 kWh/m² kilowatt hour per square meters,C,Not specified,22 kg CO₂/m²,Not specified,Gas,Yes,None,None,None,None,Non flood zone,None,"€ 1,250,000 1250000 €","€ 1,621 1621 €",No,"Mech

Based on the dataset, we've identified two primary tasks that need to be performed across multiple columns:

1. **Handling Numerical Columns:** This involves extracting numerical data and converting it to float format. It's worth noting that our initial intention was to convert integers to Integers to optimize memory usage. However, due to the presence of NaN values, which are stored as floats in pandas, this wasn't feasible at this point. Nonetheless, we'll proceed with converting them to float for now.

2. **Dealing with Binary Columns:** Many columns contain binary values, such as "Yes" and "No." We can easily convert these columns to boolean data types instead of string representations.

3. **Special Handling for Certain Columns:** Some columns, like "flood_zone_type" and "connection_to_sewer_network," also have low cardinality and should be converted to boolean values. However, their values do not align with the typical "True" and "False" boolean mapping. Instead, they require a unique dictionary mapping compared to the other boolean columns.

In [25]:
def pre_process_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocesses a DataFrame by performing various data cleaning and transformation tasks.

    Args:
        df (pandas.DataFrame): The input DataFrame to be preprocessed.

    Returns:
        pandas.DataFrame: The preprocessed DataFrame.
    """

    def extract_numbers(df: pd.DataFrame, columns: list):
        """
        Extracts numeric values from specified columns in the DataFrame.

        Args:
            df (pandas.DataFrame): The DataFrame to extract values from.
            columns (list): List of column names to extract numeric values from.

        Returns:
            pandas.DataFrame: The DataFrame with extracted numeric values.
        """
        for column in columns:
            try:
                df[column] = df[column].str.extract(r"(\d+)").astype("float32")
            except Exception as e:
                print(f"Error processing column {column}: {e}")
        return df

    def map_values(df: pd.DataFrame, columns: list):
        """
        Maps boolean values in specified columns to True, False, or None.

        Args:
            df (pandas.DataFrame): The DataFrame to map values in.
            columns (list): List of column names with boolean values to be mapped.

        Returns:
            pandas.DataFrame: The DataFrame with mapped boolean values.
        """
        for column in columns:
            try:
                df[column] = df[column].map({"Yes": True, None: False, "No": False})
            except Exception as e:
                print(f"Error processing column {column}: {e}")
        return df

    number_columns = [
        "construction_year",
        "street_frontage_width",
        "number_of_frontages",
        "covered_parking_spaces",
        "outdoor_parking_spaces",
        "living_area",
        "living_room_surface",
        "kitchen_surface",
        "bedrooms",
        "bedroom_1_surface",
        "bedroom_2_surface",
        "bedroom_3_surface",
        "bathrooms",
        "toilets",
        "surface_of_the_plot",
        "width_of_the_lot_on_the_street",
        "garden_surface",
        "primary_energy_consumption",
        "co2_emission",
        "yearly_theoretical_total_energy_consumption",
    ]

    boolean_columns = [
        "basement",
        "furnished",
        "gas_water__electricity",
        "double_glazing",
        "planning_permission_obtained",
        "tv_cable",
        "dining_room",
        "proceedings_for_breach_of_planning_regulations",
        "subdivision_permit",
        "tenement_building",
        "possible_priority_purchase_right",
    ]

    return (
        df.sort_index(axis=1)
        .fillna(np.nan)
        .rename(
            columns=lambda column: column.lower()
            .replace(" ", "_")
            .replace("&", "")
            .replace(",", "")
        )
        .rename(columns={"co₂_emission": "co2_emission"})
        .pipe(lambda df: extract_numbers(df, number_columns))
        .pipe(lambda df: map_values(df, boolean_columns))
        .assign(
            flood_zone_type=lambda df: df.flood_zone_type.map(
                {
                    "Non flood zone": False,
                    "No": False,
                    "Possible flood zone": True,
                }
            ),
            connection_to_sewer_network=lambda df: df.connection_to_sewer_network.map(
                {
                    "Connected": True,
                    "Not connected": False,
                }
            ),
            as_built_plan=lambda df: df.as_built_plan.map(
                {
                    "Yes, conform": True,
                    "No": False,
                }
            ),
            cadastral_income=lambda df: df.cadastral_income.str.split(" ", expand=True)[
                3
            ].astype("float32"),
            price=lambda df: df.price.str.rsplit(" ", expand=True, n=2)[1].astype(
                float
            ),
        )
    )


df_pre_processed = pre_process_dataframe(df)
df_pre_processed.head().style.set_sticky(axis=0)

,address,as_built_plan,available_as_of,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,building_condition,co2_emission,cadastral_income,connection_to_sewer_network,construction_year,covered_parking_spaces,dining_room,double_glazing,energy_class,external_reference,flood_zone_type,furnished,garden_surface,gas_water__electricity,heating_type,kitchen_surface,kitchen_type,latest_land_use_designation,living_area,living_room_surface,number_of_frontages,office,outdoor_parking_spaces,planning_permission_obtained,possible_priority_purchase_right,price,primary_energy_consumption,proceedings_for_breach_of_planning_regulations,reference_number_of_the_epc_report,street_frontage_width,subdivision_permit,surface_of_the_plot,surroundings_type,tv_cable,tenement_building,toilets,website,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,ad_url,day_of_retrieval
0,Stationstraat 30 9600 - Ronse,nan,After signing the deed,True,1.000000,18.000000,13.000000,9.000000,4.000000,Just renovated,9784.000000,1279.000000,True,1971.000000,2.000000,nan,True,D,5411439,False,False,485.000000,True,Gas,16.000000,Hyper equipped,"Living area (residential, urban or rural)",197.000000,47.000000,4.000000,nan,5.000000,True,True,465000.000000,371.000000,False,1-RES,12.000000,False,992.000000,"Living area (residential, urban or rural)",True,False,2.000000,nan,20.000000,nan,https://www.immoweb.be/en/classified/villa/for-sale/ronse/9600/10838435,2023-09-23 17:11:29.870179
1,Rue de la Wallonie 2A 4680 - Oupeye,nan,After signing the deed,True,1.000000,17.000000,11.000000,nan,2.000000,To renovate,173.000000,689.000000,True,1949.000000,1.000000,True,True,G,5534704,False,False,315.000000,nan,Fuel oil,13.000000,Semi equipped,"Living area (residential, urban or rural)",139.000000,10.000000,3.000000,Yes,nan,nan,False,150000.000000,699.000000,False,20230303013078,18.000000,False,413.000000,Isolated,True,False,1.000000,http://www.nigel-immo.be,18.000000,96913.000000,https://www.immoweb.be/en/classified/house/for-sale/esneux%20tilff/4130/10838582,2023-09-23 17:11:29.995637
2,Sint-Denijslaan 1 9000 - Gent,False,After signing the deed,True,1.000000,22.000000,21.000000,13.000000,3.000000,Good,9544.000000,917.000000,True,1920.000000,nan,nan,True,C,5531386,False,False,55.000000,True,Gas,13.000000,Installed,"Living area (residential, urban or rural)",200.000000,26.000000,2.000000,Yes,nan,nan,False,529000.000000,239.000000,False,2281999,5.000000,False,136.000000,Urban,True,False,1.000000,http://www.immodavinci.be,nan,nan,https://www.immoweb.be/en/classified/mansion/for-sale/gent/9000/10835799,2023-09-23 17:11:32.203024
3,Hoogstraat 20 9340 - Lede,nan,After signing the deed,nan,nan,nan,nan,nan,3.000000,To renovate,nan,1623.000000,True,1937.000000,nan,nan,True,E,5535368,nan,False,nan,True,Gas,nan,Semi equipped,"Living area (residential, urban or rural)",230.000000,nan,2.000000,Yes,nan,True,False,245000.000000,496.000000,False,20230804-0002955788-KNR-1,8.000000,False,154.000000,Urban,True,False,2.000000,http://www.immoderas.be,8.000000,nan,https://www.immoweb.be/en/classified/house/for-sale/sint-lievens-houtem/9520/10835368,2023-09-23 17:11:34.716784
4,"Mechelsesteenweg,157 2018 - Antwerpen",nan,Depending on the tenant,True,5.000000,20.000000,18.000000,18.000000,8.000000,Good,22.000000,1621.000000,True,1900.000000,nan,nan,True,C,5534431,False,False,36.000000,True,Gas,nan,Installed,nan,360.000000,34.000000,2.000000,nan,nan,nan,nan,1250000.000000,249.000000,nan,Not specified,6.000000,nan,173.000000,Urban,True,False,6.000000,nan,6.000000,nan,https://www.immoweb.be/en/classified/mansion/for-sale/antwerp/2018/10835944,2023-09-23 17:11:34.857664


Another crucial task on our agenda is to thoroughly parse the address information. This step is vital for extracting key details such as the city, ZIP code, house number, and street. It's important to mention that as part of streamlining our dataset and eliminating redundancy, we have removed the original address field. This address parsing process will enhance the quality and usability of our data.

In [26]:
def separate_address(df: pd.DataFrame) -> pd.DataFrame:
    """Separates the address into city, street name, house number, and zip code.

    Args:
        df (pd.DataFrame): The DataFrame containing the address column.

    Returns:
        pd.DataFrame: The DataFrame with the address separated into different columns.
    """
    # Define a regular expression pattern to extract street, house number, and zip code
    pattern = r"(?P<street_name>.*?)\s*(?P<house_number>\d+\w*)?\s*(?P<zip>\d{4})"

    try:
        return df.assign(
            city=lambda df: df.address.str.rsplit("-", expand=True, n=1)[1],
            **(lambda dfx: dfx.rename(columns={"address": "original_address"}))(
                df["address"].str.extract(pattern)
            ),
            street=lambda df: df.street_name.str.replace(
                r"[^a-zA-Z\s]", "", regex=True
            ),
        ).drop(columns=["street_name", "address"])
    except Exception as e:
        print(f"Error separating address: {e}")
        return df


finer_pre_cleaned = separate_address(df_pre_processed)
finer_pre_cleaned.head().style.set_sticky(axis=0)

,as_built_plan,available_as_of,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,building_condition,co2_emission,cadastral_income,connection_to_sewer_network,construction_year,covered_parking_spaces,dining_room,double_glazing,energy_class,external_reference,flood_zone_type,furnished,garden_surface,gas_water__electricity,heating_type,kitchen_surface,kitchen_type,latest_land_use_designation,living_area,living_room_surface,number_of_frontages,office,outdoor_parking_spaces,planning_permission_obtained,possible_priority_purchase_right,price,primary_energy_consumption,proceedings_for_breach_of_planning_regulations,reference_number_of_the_epc_report,street_frontage_width,subdivision_permit,surface_of_the_plot,surroundings_type,tv_cable,tenement_building,toilets,website,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,ad_url,day_of_retrieval,city,house_number,zip,street
0,nan,After signing the deed,True,1.000000,18.000000,13.000000,9.000000,4.000000,Just renovated,9784.000000,1279.000000,True,1971.000000,2.000000,nan,True,D,5411439,False,False,485.000000,True,Gas,16.000000,Hyper equipped,"Living area (residential, urban or rural)",197.000000,47.000000,4.000000,nan,5.000000,True,True,465000.000000,371.000000,False,1-RES,12.000000,False,992.000000,"Living area (residential, urban or rural)",True,False,2.000000,nan,20.000000,nan,https://www.immoweb.be/en/classified/villa/for-sale/ronse/9600/10838435,2023-09-23 17:11:29.870179,Ronse,30,9600,Stationstraat
1,nan,After signing the deed,True,1.000000,17.000000,11.000000,nan,2.000000,To renovate,173.000000,689.000000,True,1949.000000,1.000000,True,True,G,5534704,False,False,315.000000,nan,Fuel oil,13.000000,Semi equipped,"Living area (residential, urban or rural)",139.000000,10.000000,3.000000,Yes,nan,nan,False,150000.000000,699.000000,False,20230303013078,18.000000,False,413.000000,Isolated,True,False,1.000000,http://www.nigel-immo.be,18.000000,96913.000000,https://www.immoweb.be/en/classified/house/for-sale/esneux%20tilff/4130/10838582,2023-09-23 17:11:29.995637,Oupeye,2A,4680,Rue de la Wallonie
2,False,After signing the deed,True,1.000000,22.000000,21.000000,13.000000,3.000000,Good,9544.000000,917.000000,True,1920.000000,nan,nan,True,C,5531386,False,False,55.000000,True,Gas,13.000000,Installed,"Living area (residential, urban or rural)",200.000000,26.000000,2.000000,Yes,nan,nan,False,529000.000000,239.000000,False,2281999,5.000000,False,136.000000,Urban,True,False,1.000000,http://www.immodavinci.be,nan,nan,https://www.immoweb.be/en/classified/mansion/for-sale/gent/9000/10835799,2023-09-23 17:11:32.203024,Gent,1,9000,SintDenijslaan
3,nan,After signing the deed,nan,nan,nan,nan,nan,3.000000,To renovate,nan,1623.000000,True,1937.000000,nan,nan,True,E,5535368,nan,False,nan,True,Gas,nan,Semi equipped,"Living area (residential, urban or rural)",230.000000,nan,2.000000,Yes,nan,True,False,245000.000000,496.000000,False,20230804-0002955788-KNR-1,8.000000,False,154.000000,Urban,True,False,2.000000,http://www.immoderas.be,8.000000,nan,https://www.immoweb.be/en/classified/house/for-sale/sint-lievens-houtem/9520/10835368,2023-09-23 17:11:34.716784,Lede,20,9340,Hoogstraat
4,nan,Depending on the tenant,True,5.000000,20.000000,18.000000,18.000000,8.000000,Good,22.000000,1621.000000,True,1900.000000,nan,nan,True,C,5534431,False,False,36.000000,True,Gas,nan,Installed,nan,360.000000,34.000000,2.000000,nan,nan,nan,nan,1250000.000000,249.000000,nan,Not specified,6.000000,nan,173.000000,Urban,True,False,6.000000,nan,6.000000,nan,https://www.immoweb.be/en/classified/mansion/for-sale/antwerp/2018/10835944,2023-09-23 17:11:34.857664,Antwerpen,157,2018,Mechelsesteenweg


# Assessing Feature Cardinality

We'll now examine the feature cardinality of our dataset to distinguish between categorical and numerical data. This analysis will help us categorize and work with these features effectively.

In [5]:
# Assuming df is your DataFrame
number_unique_entries = {
    "column_name": finer_pre_cleaned.columns.tolist(),
    "column_dtype": [finer_pre_cleaned[col].dtype for col in finer_pre_cleaned.columns],
    "unique_values_pct": [
        finer_pre_cleaned[col].nunique() for col in finer_pre_cleaned.columns
    ],
}

(
    pd.DataFrame(number_unique_entries)
    .sort_values("unique_values_pct")
    .assign(
        unique_values_pct=lambda x: x.unique_values_pct.div(df.shape[0])
        .mul(100)
        .round(1)
    )
    .pipe(
        lambda df: ggplot(df, aes("unique_values_pct", "column_name"))
        + geom_bar(stat="identity", orientation="y")
        + labs(
            title="Assessing Feature Cardinality",
            subtitle=""" Features with a Low Cardinality (Less than 10 Distinct Values) Can Be  Utilized as Categorical Variables, 
            while Those with Higher Cardinality, typically represented as floats or integers, May Be Employed as They Are
            """,
            x="Percentage of Unique Values per Feature",
            y="",
            caption="https://www.immoweb.be/",
        )
        + theme(
            plot_subtitle=element_text(
                size=12, face="italic"
            ),  # Customize subtitle appearance
            plot_title=element_text(size=15, face="bold"),  # Customize title appearance
        )
        + ggsize(800, 1000)
    )
)

In [6]:
(
    pd.DataFrame(number_unique_entries)
    .sort_values("unique_values_pct")
    .pipe(
        lambda df: ggplot(df, aes("unique_values_pct", "column_name"))
        + geom_bar(stat="identity", orientation="y")
        + labs(
            title="Assessing Feature Cardinality",
            subtitle=""" Features with a Low Cardinality (Less than 10 Distinct Values) Can Be  Utilized as Categorical Variables, 
            while Those with Higher Cardinality, typically represented as floats or integers, May Be Employed as They Are
            """,
            x="Number of Unique Values per Feature",
            y="",
            caption="https://www.immoweb.be/",
        )
        + theme(
            plot_subtitle=element_text(
                size=12, face="italic"
            ),  # Customize subtitle appearance
            plot_title=element_text(size=15, face="bold"),  # Customize title appearance
        )
        + ggsize(800, 1000)
    )
)